<a href="https://colab.research.google.com/github/9Ten/9Ten-ml-risk-management-with-rai/blob/main/xgboost_stock_prediction_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance responsibleai raiwidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import yfinance as yf

# Download historical data for ADVANC.BK stock
data = yf.download('ADVANC.BK')

# Create time series features
data['day_of_week'] = data.index.dayofweek
data['quarter'] = data.index.quarter
data['year'] = data.index.year
data['month'] = data.index.month
data['day_of_year'] = data.index.dayofyear

# Create lag features
num_lags = 5
for lag in range(1, num_lags + 1):
    data[f'lag_{lag}'] = data['Close'].shift(lag)

# Drop any rows with NaN values
data = data.dropna()

# Assume that 'Close' is the target variable and the rest are features
X = data.drop('Close', axis=1)
y = data['Close']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = XGBRegressor(n_estimators=1000, early_stopping_rounds=50, learning_rate=0.01, objective='reg:squarederror')

# Train the model
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)],verbose=100)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

[*********************100%%**********************]  1 of 1 completed
[0]	validation_0-rmse:68.19661	validation_1-rmse:68.53163
[100]	validation_0-rmse:25.17636	validation_1-rmse:25.26890
[200]	validation_0-rmse:9.36285	validation_1-rmse:9.37293
[300]	validation_0-rmse:3.61218	validation_1-rmse:3.63645
[400]	validation_0-rmse:1.63094	validation_1-rmse:1.74793
[500]	validation_0-rmse:1.06070	validation_1-rmse:1.28048
[600]	validation_0-rmse:0.91157	validation_1-rmse:1.18729
[700]	validation_0-rmse:0.85309	validation_1-rmse:1.16353
[798]	validation_0-rmse:0.81704	validation_1-rmse:1.16024
RMSE: 1.159706


In [ ]:
target_feature = 'y_price'

train_data = X_train.copy()
test_data = X_test.copy()
train_data[target_feature] = y_train
test_data[target_feature] = y_test
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'day_of_week',
       'quarter', 'year', 'month', 'day_of_year', 'lag_1', 'lag_2', 'lag_3',
       'lag_4', 'lag_5'],
      dtype='object')

In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights
from responsibleai.feature_metadata import FeatureMetadata
feature_metadata = FeatureMetadata(categorical_features=[], dropped_features=[])


test_data_sample = test_data.sample(n=100, random_state=5)
train_data_sample = train_data.sample(n=1000, random_state=5)

rai_insights = RAIInsights(model, train_data_sample, test_data_sample, target_feature, 'regression', feature_metadata=feature_metadata)

/usr/local/lib/python3.10/dist-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/usr/local/lib/python3.10/dist-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swap_size=100, num_

In [ ]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()

# Counterfactuals: accepts total number of counterfactuals to generate, the range that their label should fall under,
# and a list of strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=20, desired_range=[50, 120])

In [ ]:
rai_insights.compute()

Causal Effects
Current Status: Generating Causal Effects.
Current Status: Finished generating causal effects.
Time taken: 0.0 min 9.630900001411646e-05 sec
Counterfactual
Current Status: Generating 20 counterfactuals for 100 samples


100%|██████████| 100/100 [00:56<00:00,  1.78it/s]


Current Status: Generated 20 counterfactuals for 100 samples.
Time taken: 0.0 min 57.88103938199998 sec
Error Analysis
Current Status: Generating error analysis reports.
Current Status: Finished generating error analysis reports.
Time taken: 0.0 min 0.10517205600001489 sec
Explanations
Current Status: Explaining 15 features
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000255 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2823
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 15
[LightGBM] [Info] Start training from score 136.996102
Current Status: Explained 15 features.
Time taken: 0.0 min 1.3602107400000136 sec


In [ ]:
ResponsibleAIDashboard(rai_insights)

ResponsibleAI started at http://localhost:8704


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.32.147.68


In [14]:
!npx localtunnel --port 8704

npx: installed 22 in 6.841s
your url is: https://ten-numbers-leave.loca.lt
^C
